In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense,RepeatVector
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D,Conv2D
from tensorflow.keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
import tensorflow
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import  SimpleRNN
from statsmodels.tools.eval_measures import rmse 
from fbprophet import Prophet

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

In [3]:
def x_y_split(df,ratio):
    interval=int(len(df)*ratio)
    train = df[:interval]
    test = df[interval:]
    train.dropna(inplace=True)
    test.dropna(inplace=True)
    return train,test

In [4]:
def convert2matrix(data_arr, look_back):
 X, Y =[], []
 for i in range(len(data_arr)-look_back):
  d=i+look_back  
  X.append(data_arr[i:d,0])
  Y.append(data_arr[d,0])
 return np.array(X), np.array(Y)

In [5]:
def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model

In [6]:
def model_cnn(xtrain):
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(xtrain.shape[1], 1)))
    model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [7]:
def model_lstm(xtrain):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(xtrain.shape[1], xtrain.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [8]:
def model_rnn(look_back):
  model=Sequential()
  model.add(SimpleRNN(units=32, input_shape=(look_back,1), activation="relu"))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
  return model
    

In [9]:
def reshape3d(data):
    return data.reshape(data.shape[0],data.shape[1],1)

In [10]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [11]:
def preprocess_weather(df,scaler):
    df=df[['temperatureMax']]
    scaler=scaler.fit(df)
    df['temperatureMax']=scaler.transform(df)
    return df

In [12]:
def preprocessing_walmart(df,scaler):
    df=df.drop(columns=['Store','Dept','IsHoliday'])
    scaler=scaler.fit(df)
    df['Weekly_Sales']=scaler.transform(df)
    return df
    

In [13]:
def preprocess_grocery(df,scaler):
    df.drop(['id','store_nbr','item_nbr','onpromotion'], axis=1,inplace=True)
    scaler=scaler.fit(df)
    df['unit_sales']=scaler.transform(df)
    return df

In [29]:
import os
import logging, sys
logging.disable(sys.maxsize)
from math import sqrt
paths_to_folders = ['C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Practice/weather']
x=0
scaler = MinMaxScaler(feature_range = (0, 1))
warnings.filterwarnings("ignore")
# error = pd.DataFrame(columns=['File_name', 'ARIMA_RMSE_ERROR','SES_RMSE_ERROR','SARIMA_RMSE_ERROR','PROPHET_RMSE_ERROR',
#                               'HOLT_WINTER_RMSE_ERROR','ARIMA_MAPE_ERROR','SES_MAPE_ERROR','SARIMA_MAPE_ERROR','PROPHET_MAPE_ERROR',
#                               'HOLT_WINTER_MAPE_ERROR'])
error = pd.DataFrame(columns=['File_name','PROPHET+CNN_MAPE', 'PROPHET+DNN_MAPE','PROPHET+RNN_MAPE','PROPHET+CNN_RMSE','PROPHET+DNN_RMSE','PROPHET+RNN_RMSE'])
for folder in paths_to_folders:
   for csv_file in os.listdir(folder):
#      df=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Practice/weather"+csv_file,parse_dates=['Date'],index_col='Date')
     df=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Practice/weather/"+csv_file,parse_dates=['time'],index_col='time')
     if len(df.index)>=100:
         test_predictions=pd.DataFrame()
         df=preprocess_weather(df,scaler)
         train,test=x_y_split(df,0.8)
         interval=int(len(df)*0.8)
        
         train_prophet=pd.DataFrame()
         train_prophet['ds']=train.index
         train_prophet['y']=np.array(train['temperatureMax'])
        
         m1 = Prophet(daily_seasonality=True)
         m1.fit(train_prophet)
         future1 = m1.make_future_dataframe(periods=len(test.index),freq='D')
         forecast1 = m1.predict(future1)
         pred=forecast1[['yhat']]
         p=pred.iloc[interval:]
         actual=np.array(p)
         actual=actual.reshape(-1)
         df_res=pd.DataFrame(columns=['y'])
         df_res=pd.DataFrame(np.array(df['temperatureMax'])-np.array(pred['yhat']))
         df_res['y']=df_res
         df_res=df_res.loc[:,['y']]
            
         train_res,test_res=x_y_split(df_res,0.8)
         
         
         timestep=4
         xtrain,ytrain=convert2matrix(train_res.values,timestep)
         xtest,ytest=convert2matrix(test_res.values,timestep)
         test_predictions['actual']=np.array(ytest)
        
         #DNN MODEL
         model = model_dnn(timestep)
         model.fit(xtrain,ytrain, epochs=60, batch_size=32, verbose=1,callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)
         test_predict = model.predict(xtest)
         test_predictions['DNN']=np.array(test_predict)
            
        
         #RNN Model  
         model = model_rnn(timestep)
         model.fit(reshape3d(xtrain),ytrain,epochs=100, batch_size=30, verbose=1,callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)
         test_predict = model.predict(reshape3d(xtest))
         test_predictions['RNN']=np.array(test_predict)
        
            
         #CNN MODEL
         model=model_cnn(reshape3d(xtrain))
         model.fit(reshape3d(xtrain), ytrain, epochs=60, verbose=1)
         cnn_preds = model.predict(reshape3d(xtest))
         test_predictions['CNN']=np.array(cnn_preds)
         
       
        
        
         error.loc[x, ['File_name']]=csv_file
         error.loc[x, ['PROPHET+DNN_MAPE']]=mean_absolute_percentage_error(np.array(test_predictions['DNN'])+actual[timestep:],np.array(test[timestep:]['temperatureMax']))
         error.loc[x, ['PROPHET+CNN_MAPE']]=mean_absolute_percentage_error(np.array(test_predictions['CNN'])+actual[timestep:],np.array(test[timestep:]['temperatureMax']))
         error.loc[x, ['PROPHET+RNN_MAPE']]=mean_absolute_percentage_error(np.array(test_predictions['RNN'])+actual[timestep:],np.array(test[timestep:]['temperatureMax']))
        
        
         error.loc[x, ['PROPHET+DNN_RMSE']]=rmse(np.array(test_predictions['DNN'])+actual[timestep:],np.array(test[timestep:]['temperatureMax']))
         error.loc[x, ['PROPHET+CNN_RMSE']]=rmse(np.array(test_predictions['CNN'])+actual[timestep:],np.array(test[timestep:]['temperatureMax']))
         error.loc[x, ['PROPHET+RNN_RMSE']]=rmse(np.array(test_predictions['RNN'])+actual[timestep:],np.array(test[timestep:]['temperatureMax']))
         
       

        
        
         x=x+1
         print(csv_file)
         print(x)
         #print(rmse(test["unit_sales"], predictions))
         #print(mean_squared_error(test["unit_sales"], predictions))

Epoch 1/60
85/85 [==============================] - 0s 3ms/step - loss: 0.0167 - mse: 0.0167 - mae: 0.0961
Epoch 2/60
85/85 [==============================] - 0s 188us/step - loss: 0.0155 - mse: 0.0155 - mae: 0.0916
Epoch 3/60
85/85 [==============================] - 0s 282us/step - loss: 0.0145 - mse: 0.0145 - mae: 0.0886
Epoch 4/60
85/85 [==============================] - 0s 188us/step - loss: 0.0137 - mse: 0.0137 - mae: 0.0857
Epoch 5/60
85/85 [==============================] - 0s 188us/step - loss: 0.0129 - mse: 0.0129 - mae: 0.0833
Epoch 6/60
85/85 [==============================] - 0s 282us/step - loss: 0.0123 - mse: 0.0123 - mae: 0.0811
Epoch 7/60
85/85 [==============================] - 0s 94us/step - loss: 0.0117 - mse: 0.0117 - mae: 0.0794
Epoch 8/60
85/85 [==============================] - 0s 282us/step - loss: 0.0112 - mse: 0.0112 - mae: 0.0779
Epoch 9/60
85/85 [==============================] - 0s 94us/step - loss: 0.0108 - mse: 0.0108 - mae: 0.0765
Epoch 10/60
85/85 [====

Epoch 88/100
85/85 [==============================] - 0s 188us/step - loss: 0.0036 - mse: 0.0036 - mae: 0.0459
Epoch 89/100
85/85 [==============================] - 0s 188us/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0456
Epoch 90/100
85/85 [==============================] - 0s 188us/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0456
Epoch 91/100
85/85 [==============================] - 0s 188us/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0454
Epoch 92/100
85/85 [==============================] - 0s 282us/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0452
Epoch 93/100
85/85 [==============================] - 0s 188us/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0452
Epoch 94/100
85/85 [==============================] - 0s 188us/step - loss: 0.0034 - mse: 0.0034 - mae: 0.0452
Epoch 95/100
85/85 [==============================] - 0s 188us/step - loss: 0.0034 - mse: 0.0034 - mae: 0.0449
Epoch 96/100
85/85 [==============================] - 0s 188us/step - loss: 0.0034 - mse: 0.0034 - mae: 0.0447
E

In [30]:
error

,File_name,PROPHET+CNN_MAPE,PROPHET+DNN_MAPE,PROPHET+RNN_MAPE,PROPHET+CNN_RMSE,PROPHET+DNN_RMSE,PROPHET+RNN_RMSE
0,Armenia.csv,13.3003,12.5559,13.6241,0.116647,0.106376,0.116888


In [31]:
error.to_csv('weather-Hybrid-model.csv')

In [ ]:
pd.DataFrame(np.array(test_predictions['actual'])+np.array(test[4:]['Weekly_Sales']))

In [55]:
rmse(np.array(test_predictions['DNN'])+actual[timestep:],np.array(test[timestep:]['Weekly_Sales']))

0.17547138038238477

In [56]:
np.array(test_predictions['DNN'])+actual[timestep:]

array([0.51207182, 0.5660495 , 0.51820978, 0.59229423, 0.51694085,
       0.49243493, 0.50176779, 0.53905422, 0.55195408, 0.50873519,
       0.51976303, 0.51909784, 0.54605435, 0.485287  , 0.46796   ,
       0.48637555, 0.56506839, 0.4999135 , 0.47844099, 0.47776161,
       0.55201982, 0.58731638, 0.50694631, 0.52127355, 0.46040401])

In [57]:
np.array(test[4:]['Weekly_Sales'])

array([0.54042436, 0.66089465, 0.52082436, 0.80236486, 0.80929628,
       0.71567443, 0.69627897, 0.70469893, 0.84143192, 0.66314339,
       0.54900341, 0.624307  , 0.86845535, 0.69180877, 0.63539386,
       0.53683252, 0.7609559 , 0.71133605, 0.58070725, 0.55098852,
       0.51873925, 0.8063385 , 0.53399178, 0.73035767, 0.62190941])

In [58]:
actual[timestep:]

array([0.53244452, 0.53301856, 0.53294748, 0.53222145, 0.53085533,
       0.52888806, 0.5263813 , 0.52341759, 0.52009773, 0.51653784,
       0.51286591, 0.50921811, 0.50573481, 0.50255664, 0.49982044,
       0.49765552, 0.49617999, 0.49549757, 0.49569475, 0.49683847,
       0.49897443, 0.50212586, 0.50629301, 0.51145327, 0.51756182])